In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import pandas as pd
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
from scipy.stats import skew,boxcox_normmax
from scipy.special import boxcox1p
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
# % matplotlib inline
sns.set_style('dark')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# train=pd.read_csv('../input/tabular-playground-series-may-2021/train.csv',index_col='id')
test=pd.read_csv('../input/tabular-playground-series-may-2021/test.csv',index_col='id')
train.head()

In [ ]:
print(test.shape,train.shape)

In [ ]:
# train=train.astype(str)
# test=test.astype(str)
# # train.info()
cat_col=['target']
train_df=pd.get_dummies(train,columns=cat_col)
train_df.head()

In [ ]:
target=train['target']
# dup_combo=pd.concat([train.drop('target',axis=1),test],keys=['train','test'],axis=0)

In [ ]:
# cat_col=dup_combo.select_dtypes(include=['object'])
# te=pd.get_dummies(cat_col)
# dup_combo=pd.concat([dup_combo,te],axis=1)
# dup_cleaned = dup_combo.drop(cat_col.columns,axis=1)
# dup_cleaned.shape

In [ ]:
# Testing Data
X_test=train.drop('target',axis=1)[:1000]
y_test=target[:1000]

X=train.iloc[1000:,:50]
y=train_df.iloc[1000:,50:55]

In [ ]:
y.shape

In [ ]:
# X,test=X.xs('train'),X.xs('test')

In [ ]:
# scalar=StandardScaler()
# scalar=scalar.fit(X)
# X=scalar.transform(X)

In [ ]:
X.values

In [ ]:
for i in range(len(y_test)):
    y_test[i]=y_test[i].split('_')[-1]
    
# for i in range(1000,100000):
#     y[i]=y[i].split('_')[-1]

In [ ]:
X_train,X_valid,y_train,y_valid=train_test_split(X,y,test_size=0.2,stratify=y,random_state=101)

In [ ]:
y_train.sum()

In [ ]:
sampling_dict={0:6707,1:25000,2:16962,3:9972}

under_sample=RandomUnderSampler(sampling_strategy=sampling_dict)

X_train,y_train = under_sample.fit_resample(X_train.values,y_train.values)

In [ ]:
# # y_valid.unique()
# y_train=y_train.map({'1':0,'2':1,'3':2,'4':3})
# y_valid=y_valid.map({'1':0,'2':1,'3':2,'4':3})
y_test=y_test.map({'1':0,'2':1,'3':2,'4':3})

# type(y_train[57850])
y_test.values

In [ ]:
# y_train=keras.utils.to_categorical(y_train,num_classes=4)
# y_valid=keras.utils.to_categorical(y_valid,num_classes=4)
y_test=keras.utils.to_categorical(y_test,num_classes=4)
y_test

In [ ]:
print(X_train.shape,y_train.shape)

In [ ]:
model=keras.Sequential([
    layers.Input(shape=X_train.shape[1]),
    
#     layers.Dropout(0.3),
#     layers.BatchNormalization(),
#     layers.Dense(240,activation='relu'),
    
#     layers.Dropout(0.3),
#     layers.BatchNormalization(),
#     layers.Dense(240,activation='relu'),
    
#     layers.Dropout(0.5),
#     layers.BatchNormalization(),
#     layers.Dense(720,activation='relu'),
    
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(360,activation='relu'),
    
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(180,activation='relu'),
    
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(90,activation='relu'),
    
#     layers.Dropout(0.5),
#     layers.BatchNormalization(),
#     layers.Dense(30,activation='relu'),
    
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(30,activation='relu'),
    
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(20,activation='relu'),
    
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(20,activation='relu'),
    
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(20,activation='relu'),
    
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(10,activation='relu'),
    
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(10,activation='relu'),
    
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(10,activation='relu'),
    
    layers.Dense(4,activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# early_stop=EarlyStopping(
#     patience=5,
#     min_delta=0.0001,
#     restore_best_weights=True
# )

history=model.fit(
    X_train,y_train,
    validation_data=(X_valid,y_valid),
    batch_size=256,
    epochs=60,
#     callbacks=[early_stop]
)

In [ ]:
acc=pd.DataFrame(history.history)
acc[['loss','val_loss']].plot()
acc[['accuracy','val_accuracy']].plot()

In [ ]:
print(model.evaluate(X_test,y_test))
# X_test.shape

In [ ]:
y_test

In [ ]:
# print(classification_report(y_test,model.predict(X_test)))
# model.predict(X_test)
# y_test

In [ ]:
test=scaler.fit_transform(test)

In [ ]:
pred=model.predict(X_valid)
pred

In [ ]:
ans=model.predict(test)

In [ ]:
pd.DataFrame(ans)

In [ ]:
dup=pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')

In [ ]:
answer=pd.DataFrame(ans,columns=['Class_1','Class_2','Class_3','Class_4'])

answer['id']=dup['id']
answer.set_index('id',inplace=True)

In [ ]:
answer.to_csv('playground_may')